# Part 2: Attribute Segmentation and Filtering

We can import our preprocessed data

In [6]:
#packages
import numpy as np
import pandas as pd
import datetime as dt
from datetime import timedelta
import geopandas
import bokeh

df = pd.read_csv("sales_df.csv")
df.rename(columns={ df.columns[0]: "index" }, inplace = True)
df = df.set_index("index")
df['transaction date'] = pd.to_datetime(df['transaction date'])

We have two attributes: skuID, which can be premium or unlockcharactermanager, and day of the week, to see if we get more sales in the weekends or in the weekdays. So in total we have 4 information combinations:
- premium + weekdays
- premium + weekend
- unlockchar + weekdays
- unlockchar + weekend

ofcourse, we can also look at individual days. Then we would have
- premium + mon
- premium + tue
- ...
- premium + sun
- unlockchar + mon
- unlockchar + tue
- ...
- unlockchar + sun

In [2]:
df

,transaction date,transaction type,product id,sku id,buyer country,buyer postal code,amount (merchant currency)
index,,,,,,,
0,2021-06-01,Charge,com.vansteinengroentjes.apps.ddfive,unlockcharactermanager,US,62011,4.490000
1,2021-06-01,Charge,com.vansteinengroentjes.apps.ddfive,premium,US,55320,2.860000
2,2021-06-02,Charge,com.vansteinengroentjes.apps.ddfive,unlockcharactermanager,US,54220,4.510000
3,2021-06-02,Charge,com.vansteinengroentjes.apps.ddfive,unlockcharactermanager,US,78250,4.500000
4,2021-06-02,Charge,com.vansteinengroentjes.apps.ddfive,premium,US,74830,2.860000
...,...,...,...,...,...,...,...
1694,2021-12-31,Charge,com.vansteinengroentjes.apps.ddfive,premium,US,98856,3.355112
1695,2021-12-31,Charge,com.vansteinengroentjes.apps.ddfive,unlockcharactermanager,US,98856,5.271058
1696,2021-12-31,Charge,com.vansteinengroentjes.apps.ddfive,premium,GB,NaN,3.082306


In [41]:
#make a new column with the date of the week
df['day of week'] = df['transaction date'].dt.weekday #monday is 0
df_prem = df.loc[df['sku id'] == "premium"]
df_prem_month6 = df_prem.loc[df_prem['transaction date'].dt.month == 6]
df_prem_month6

,transaction date,transaction type,product id,sku id,buyer country,buyer postal code,amount (merchant currency),day of week
index,,,,,,,,
1,2021-06-01,Charge,com.vansteinengroentjes.apps.ddfive,premium,US,55320,2.86,1
4,2021-06-02,Charge,com.vansteinengroentjes.apps.ddfive,premium,US,74830,2.86,2
5,2021-06-02,Charge,com.vansteinengroentjes.apps.ddfive,premium,IL,7548207,3.25,2
7,2021-06-02,Charge,com.vansteinengroentjes.apps.ddfive,premium,SE,226 48,1.72,2
8,2021-06-02,Charge,com.vansteinengroentjes.apps.ddfive,premium,US,74429,2.86,2
...,...,...,...,...,...,...,...,...
214,2021-06-28,Charge,com.vansteinengroentjes.apps.ddfive,premium,US,68873,2.93,0
217,2021-06-29,Charge,com.vansteinengroentjes.apps.ddfive,premium,US,85710,2.93,1
218,2021-06-30,Charge,com.vansteinengroentjes.apps.ddfive,premium,US,84057,2.95,2


In [39]:
#first we want a diagram that is a plot of the cumulative graph of the sales volume (amounts for now, might add a tab with money later)
#the plot has two lines: unlockcharactermanager and premium.

#we want a list of the cumulative data per date. We already did this in the last assignment:
#first the month labels
months = ["Jun","Jul","Aug","Sep","Oct","Nov","Dec"]
month_map = {
    "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, "May": 5, "Jun": 6,
    "Jul": 7, "Aug": 8, "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12
}#to convert name to number
#now we need the monthly sums, this will be the sum of data rows
monthly_sums = []
# Daily data for each month
daily_data = {#days of each relevant month
    "Jun": [str(i) for i in range(1, 31)],
    "Jul": [str(i) for i in range(1, 32)],
    "Aug": [str(i) for i in range(1, 32)],
    "Sep": [str(i) for i in range(1, 31)],
    "Oct": [str(i) for i in range(1, 32)],
    "Nov": [str(i) for i in range(1, 31)],
    "Dec": [str(i) for i in range(1, 32)]
}
#the initial plot can use dictionary input: we will process that here
daily_values_premium = {}
daily_values_unlock = {}
for month in months:
    #get all the dates of this month 
    month_values = df.loc[df['transaction date'].dt.month == month_map[month]]
    #get values for premium and unlock separately
    prem_values = month_values.loc[month_values['sku id'] == "premium"]
    unlock_values = month_values.loc[month_values['sku id'] == "unlockcharactermanager"]
    #now we iterate over the days and append the length of a series of only that day
    prem_day_value_list = []
    unlock_day_value_list = []
    for i in daily_data[month]:
        prem_day_value_list.append(len(prem_values.loc[prem_values['transaction date'].dt.day==int(i)]))
        unlock_day_value_list.append(len(unlock_values.loc[unlock_values['transaction date'].dt.day==int(i)]))
    #now we need the daily values as a list to append to the daily values dictionary
    daily_values_premium[month] = prem_day_value_list
    daily_values_unlock[month] = unlock_day_value_list

#now we have a dictionary with all the daily values. Now we need to combine this with an x-axis that's
#all the dates between the start and end of our data set
flat_daily_values_premium = [value for month in months for value in daily_values_premium[month]] #will be our y-axis
flat_daily_values_unlock = [value for month in months for value in daily_values_unlock[month]] #will be our y-axis
dates_list = [df['transaction date'][0] + timedelta(days=i) for i in range(len(flat_daily_values_premium))]#we can choose 1 of the 2


# Bokeh libraries
from bokeh.io import output_notebook
from bokeh.plotting import figure, show

# Output the visualization directly in the notebook
output_notebook()

# Create a figure with a datetime type x-axis
fig = figure(title='TITLE',
             height=400, width=700,
             x_axis_label='Day Number', y_axis_label='Sales amount',
             toolbar_location=None)

# The cumulative sum will be a trend line
fig.line(x=dates_list, y=np.cumsum(flat_daily_values_premium),
         color='gray', line_width=2)
fig.line(x=dates_list, y=np.cumsum(flat_daily_values_unlock),
         color='red', line_width=2)

# Let's check it out
show(fig)

Loading BokehJS ...

In [47]:
#now we want a bar chart with average sales data for each weekday
#so the x-axis will be:
days = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]
days_map = {
    "Mon": 0, "Tue": 1, "Wed": 2, "Thu": 3, "Fri": 4, "Sat": 5,
    "Sun": 6
}
#and the y-axis contains bars with the average sales amount over the entire dataset (TITLE SHOULD REFLECT THIS)
#we can again have two tabs, one for sales data and one for sales volume

#first, we need to get a list of average amount of sales per weekday
days_sales_volume_total = {}
days_sales_volume_premium = {}
days_sales_volume_unlock = {}
for day in days:
    #get the mean amount of sales for that day
    day_df = df.loc[df['day of week'] == days_map[day]]
    total_sales_vol = len(day_df)
    premium_sales_vol = len(day_df.loc[day_df['sku id'] == "premium"])
    unlock_sales_vol = len(day_df.loc[day_df['sku id'] == "unlockcharactermanager"])
    days_sales_volume_total[day] = total_sales_vol/7 #average over all week days
    days_sales_volume_premium[day] = premium_sales_vol/7 #average over all week days
    days_sales_volume_unlock[day] = unlock_sales_vol/7 #average over all week days


###################TOTAL FIGURE###############

# Output the visualization directly in the notebook
output_notebook()

# Extract x and y values from the dictionary
x_values = list(days_sales_volume_total.keys())  # ["Mon", "Tue", ...]
y_values = list(days_sales_volume_total.values())  # Corresponding sales amounts

# Create a Bokeh data source
source = ColumnDataSource(data=dict(days=x_values, sales=y_values))

# Create a figure
fig = figure(title='Average Sales Volume per Weekday',
             height=400, width=700,
             x_axis_label='Day of Week', y_axis_label='Sales Volume',
             x_range=x_values,  # Ensure correct categorical axis
             toolbar_location=None)

# Add bars to the figure
fig.vbar(x='days', top='sales', width=0.5, source=source, 
         fill_color=factor_cmap('days', palette="Blues7", factors=x_values))

# Show the plot
show(fig)


##############PREMIUM+UNLOCK FIGURE################
# Output the visualization in the notebook
output_notebook()

# Extract x-axis labels (weekdays)
x_values = list(days_sales_volume_premium.keys())  # ["Mon", "Tue", "Wed", ...]

# Extract y-axis values (sales volume) for both categories
premium_sales = list(days_sales_volume_premium.values())
unlock_sales = list(days_sales_volume_unlock.values())

# Create a Bokeh data source
source = ColumnDataSource(data=dict(days=x_values, premium=premium_sales, unlock=unlock_sales))

# Create figure
fig = figure(title='Average Sales Volume per Weekday (Premium vs Unlock)',
             height=400, width=700,
             x_axis_label='Day of Week', y_axis_label='Sales Volume',
             x_range=x_values,  # Ensures categorical axis
             toolbar_location=None)

# Bar width and dodge distance
bar_width = 0.4  # Adjust width for better spacing
dodge_dist = 0.2  # Moves one bar left and the other right

# Add bars for Premium Sales
fig.vbar(x=dodge('days', -dodge_dist, range=fig.x_range), 
         top='premium', width=bar_width, source=source, 
         color="royalblue", legend_label="Premium Sales")

# Add bars for Unlock Sales
fig.vbar(x=dodge('days', dodge_dist, range=fig.x_range), 
         top='unlock', width=bar_width, source=source, 
         color="orange", legend_label="Unlock Sales")

# Customize legend
fig.legend.title = "Sales Type"
fig.legend.location = "top_left"

# Show plot
show(fig)


Loading BokehJS ...

Loading BokehJS ...